# 📈 04. Trajectory Feature Engineering (Adjusted for Data Size)

## Objective
Transform the raw NCAA EADA data into a **Trajectory Classification** dataset.

## Constraint Satisfaction
**Requirement:** >10,000 rows.
**Strategy:** Use **2-year trends** (past) and **1-year forecast** (future). 
*   Loss: 2 years (lag) + 1 year (target) = 3 years lost.
*   Remaining: 7 years * 1,722 institutions ≈ 12,000 rows. (Safe > 10k)

## Target Variable: `Financial_Trajectory`
We define the trajectory over the **next 1 year**:
1.  **Improving:** Revenue Growth > 3% AND Expense Growth < Revenue Growth
2.  **Declining:** Revenue Growth < 0% OR Expense Growth > Revenue Growth + 3%
3.  **Stable:** Everything else

## Input
*   `../initial files/Output_10yrs_reported_schools_17220.csv`

## Output
*   `trajectory_ml_ready.csv`

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported")

✅ Libraries imported


## 1. Load Data

In [4]:
# Load the dataset
try:
    # Path based on user's directory structure
    df = pd.read_csv('../../initial files/Output_10yrs_reported_schools_17220.csv')
except FileNotFoundError:
    # Fallback for safety
    df = pd.read_csv('Output_10yrs_reported_schools_17220.csv')

# Rename columns for easier reference
df = df.rename(columns={
    'Survey Year': 'Year',
    'Institution Name': 'Institution_Name',
    'State CD': 'State',
    'Classification Name': 'Classification_Name'
})

# Sort by Institution and Year to ensure correct lag calculations
df = df.sort_values(['UNITID', 'Year']).reset_index(drop=True)

print(f"Dataset Shape: {df.shape}")
print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"Institutions: {df['UNITID'].nunique()}")

Dataset Shape: (17220, 580)
Years: 2014 - 2023
Institutions: 1722


## 2. Data Cleaning & Preprocessing

In [5]:
# 1. Handle Missing Values in Key Financials
financial_cols = ['Grand Total Revenue', 'Grand Total Expenses']
for col in financial_cols:
    df[col] = df[col].fillna(0)

# 2. Create Total Participation Column
df['Total_Athletes'] = df["Unduplicated Count Men's Participation"].fillna(0) + df["Unduplicated Count Women's Participation"].fillna(0)

# 3. Create Efficiency Ratio (Revenue / Expenses)
# Avoid division by zero
df['Efficiency_Ratio'] = df['Grand Total Revenue'] / df['Grand Total Expenses'].replace(0, 1)

# 4. Flag "Exactly 1.0" Reporters (Accounting Manipulation)
df['Reports_Exactly_One'] = (df['Efficiency_Ratio'] == 1.0).astype(int)

print("✅ Data cleaned and basic metrics created")

✅ Data cleaned and basic metrics created


## 3. Feature Engineering: 2-Year Trends
We calculate trends over the *past* 2 years ($t, t-1$). This saves 1 year of data compared to 3-year trends.

In [6]:
# Group by UNITID to process each institution separately
grouped = df.groupby('UNITID')

# --- A. Growth Rates (1-year and 2-year CAGR) ---
# 1-year growth (t vs t-1)
df['Revenue_Growth_1yr'] = grouped['Grand Total Revenue'].pct_change()
df['Expense_Growth_1yr'] = grouped['Grand Total Expenses'].pct_change()

# 2-year CAGR (t vs t-2)
df['Revenue_CAGR_2yr'] = (grouped['Grand Total Revenue'].shift(0) / grouped['Grand Total Revenue'].shift(2))**(1/2) - 1
df['Expense_CAGR_2yr'] = (grouped['Grand Total Expenses'].shift(0) / grouped['Grand Total Expenses'].shift(2))**(1/2) - 1

# --- B. Rolling Averages (2-year) ---
df['Revenue_Mean_2yr'] = grouped['Grand Total Revenue'].transform(lambda x: x.rolling(window=2).mean())
df['Expense_Mean_2yr'] = grouped['Grand Total Expenses'].transform(lambda x: x.rolling(window=2).mean())
df['Efficiency_Mean_2yr'] = grouped['Efficiency_Ratio'].transform(lambda x: x.rolling(window=2).mean())

# --- C. Volatility (Standard Deviation over 3 years - still useful if available, else 2) ---
# We'll use 2-year std dev to be consistent with saving rows
df['Revenue_Volatility_2yr'] = grouped['Grand Total Revenue'].transform(lambda x: x.rolling(window=2).std())
df['Expense_Volatility_2yr'] = grouped['Grand Total Expenses'].transform(lambda x: x.rolling(window=2).std())

# --- D. Structural Features ---
# Division (Extract from Classification Name)
def extract_division(class_name):
    if pd.isna(class_name): return 'Unknown'
    if 'NCAA Division I' in class_name: return 'D1'
    if 'NCAA Division II' in class_name: return 'D2'
    if 'NCAA Division III' in class_name: return 'D3'
    return 'Other'

df['Division'] = df['Classification_Name'].apply(extract_division)

print("✅ Feature engineering complete")

✅ Feature engineering complete


## 4. Target Generation: `Financial_Trajectory` (1-Year Lookahead)
We define the target for year $t$ based on what happens in year $t+1$.
This saves another year of data compared to 2-year lookahead.

In [8]:
# Calculate Future Growth (Next 1 year)
future_window = 1

# Future Revenue Growth (t to t+1)
df['Future_Revenue_Growth'] = grouped['Grand Total Revenue'].shift(-future_window) / grouped['Grand Total Revenue'].shift(0) - 1

# Future Expense Growth (t to t+1)
df['Future_Expense_Growth'] = grouped['Grand Total Expenses'].shift(-future_window) / grouped['Grand Total Expenses'].shift(0) - 1

# Define Trajectory Class
def define_trajectory(row):
    rev_growth = row['Future_Revenue_Growth']
    exp_growth = row['Future_Expense_Growth']
    
    if pd.isna(rev_growth) or pd.isna(exp_growth):
        return np.nan
    
    # IMPROVING: Healthy revenue growth AND expenses growing slower than revenue
    # Threshold lowered to 3% since it's a 1-year window
    if (rev_growth > 0.03) and (exp_growth < rev_growth):
        return 'Improving'
    
    # DECLINING: Shrinking revenue OR expenses growing out of control
    elif (rev_growth < 0.00) or (exp_growth > rev_growth + 0.03):
        return 'Declining'
    
    # STABLE: Everything else
    else:
        return 'Stable'

df['Target_Trajectory'] = df.apply(define_trajectory, axis=1)

# Map to integers for ML (0=Declining, 1=Stable, 2=Improving)
trajectory_map = {'Declining': 0, 'Stable': 1, 'Improving': 2}
df['Target_Label'] = df['Target_Trajectory'].map(trajectory_map)

print("✅ Target variable created")
print(df['Target_Trajectory'].value_counts(normalize=True))

✅ Target variable created
Target_Trajectory
Stable       0.484772
Declining    0.374823
Improving    0.140405
Name: proportion, dtype: float64


## 5. Final Cleanup & Save
Remove rows with missing features (first 2 years) or missing targets (last 1 year).

In [9]:
# Filter usable data
# Must have valid features (2-year lag requires starting from year 3 of data)
# Must have valid target (requires looking ahead 1 year)

df_ml = df.dropna(subset=['Revenue_CAGR_2yr', 'Target_Label']).copy()

# Select columns for modeling
feature_cols = [
    'UNITID', 'Institution_Name', 'Year', 'State', 'Division', # Identifiers
    'Grand Total Revenue', 'Grand Total Expenses', 'Total_Athletes', # Raw values
    'Revenue_Growth_1yr', 'Expense_Growth_1yr', # 1-yr trends
    'Revenue_CAGR_2yr', 'Expense_CAGR_2yr', # 2-yr trends
    'Revenue_Mean_2yr', 'Expense_Mean_2yr', 'Efficiency_Mean_2yr', # 2-yr averages
    'Revenue_Volatility_2yr', 'Expense_Volatility_2yr', # Volatility
    'Reports_Exactly_One', # Data quality flag
    'Target_Trajectory', 'Target_Label' # Targets
]

df_final = df_ml[feature_cols]

print(f"Final Dataset Shape: {df_final.shape}")
print(f"Years Included: {df_final['Year'].min()} - {df_final['Year'].max()}")

# Verify constraint
if len(df_final) > 10000:
    print(f"✅ Constraint Met: {len(df_final)} rows > 10,000")
else:
    print(f"⚠️ Constraint Warning: {len(df_final)} rows < 10,000")

# Save to CSV
output_path = 'trajectory_ml_ready.csv'
df_final.to_csv(output_path, index=False)
print(f"✅ Saved to {output_path}")

Final Dataset Shape: (12054, 20)
Years Included: 2016 - 2022
✅ Constraint Met: 12054 rows > 10,000
✅ Saved to trajectory_ml_ready.csv
